In [6]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
import json
import plotly.graph_objs as go

In [8]:
import csv
from bs4 import BeautifulSoup

# Open the file in read mode and parse the HTML
with open('bsp.html', 'r') as f:
    contents = f.read()

soup = BeautifulSoup(contents, 'html.parser')

race_numbers = [div.text for div in soup.find_all('div', class_='race-number')]
runners = soup.find_all('div', class_='runner')

# Prepare data for CSV
data = []
for runner in runners:
    dog_number = runner.find('div', class_='number').text
    dog_name = runner.find('div', class_='name').text
    price_win = runner.find('div', class_='price win').text
    data.append([race_numbers[0], dog_number, dog_name, price_win])

# Write data to CSV
with open('bsp_today.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Race Number", "Dog Number", "Dog Name", "Price Win"])  # write header
    writer.writerows(data)  # write data

In [ ]:
df = pd.read_feather('./data/topaz_data_w_bsp_new_w_price.fth')

In [ ]:
df = df.query('state == "VIC"')

In [ ]:
prev_race_not_vic = df.query('prev_race_state != "VIC"')

In [ ]:
prev_race_not_vic.prev_race_track.value_counts()

In [ ]:
model_name  = 'VIC-holographic-cruiser-532'
epoch = ''
all_price_df = pd.read_feather(f'model_all_price/{epoch}{model_name} - all_price_df.fth')

In [ ]:
all_price_df.to_excel(f'model_all_price/{epoch}{model_name} - all_price_df.xlsx')

In [ ]:
    # model_all_price/RL40rich-brook-43 - val_all_price_df.fth model_all_price/RL40rich-brook-43 - all_price_df.fth

In [ ]:
all_price_df.columns

In [ ]:
# all_price_df[['betfair_log_loss', 'log_loss', 'label_loss']] = all_price_df[['betfair_log_loss', 'log_loss', 'label_loss']].expanding().mean()
bydate = all_price_df.groupby(['date','raceID'])[['betfair_log_loss', 'log_loss', 'label_loss']].sum().reset_index().groupby('date').mean(numeric_only=True).expanding().mean().reset_index()
# bydate = bydate.iloc[100_000:]

In [ ]:
bydate

In [ ]:
import plotly.graph_objects as go

# Create a trace for ROI
trace_roi = go.Scatter(
    x = bydate['date'],
    y = bydate['betfair_log_loss'],
    mode = 'lines',
    name = 'betfair_log_loss',
    yaxis='y1'  # This associates this trace with the first y-axis
)

# Create a second trace for the second y-axis
trace_second = go.Scatter(
    x = bydate['date'],
    y = bydate['log_loss'],  # Replace with your second data
    mode = 'lines',
    name = 'log_loss',
    yaxis='y1'  # This associates this trace with the second y-axis
)

data = [trace_roi, trace_second]

# Define layout
layout = go.Layout(
    title = 'Interactive Plot',
    xaxis = dict(title = 'Round Price'),
    yaxis = dict(title = 'Amount'),
    yaxis2=dict(
        title='Second Amount',  # Title for the second y-axis
        overlaying='y',  # This line and the line below ensure that the second y-axis is displayed on the right
        side='right'
    )
)

# Define the figure
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()

In [ ]:
model_name  = 'VIC-holographic-cruiser-532'
epoch = ''
all_price_df = pd.read_feather(f'model_all_price/{epoch}{model_name} - all_price_df.fth')
# all_price_df
all_price_df2 = pd.read_feather(f'model_all_price/{epoch}{model_name} - val_all_price_df.fth')
all_price_df = pd.concat([all_price_df, all_price_df2])

In [ ]:
bytrack = all_price_df.groupby(['date','raceID','track'])[['betfair_log_loss', 'log_loss', 'label_loss']].sum().reset_index().groupby('track').mean(numeric_only=True)

In [ ]:
all_price_df.columns

In [ ]:
all_price_df.groupby('raceID')[['pred_prob']].sum().mean()

In [ ]:
bytrack['delta'] = bytrack['betfair_log_loss'] - bytrack['log_loss']
bytrack.plot.scatter(x='betfair_log_loss', y='delta')

In [ ]:
all_price_df.columns

In [ ]:
all_price_df.simple.sum()

In [ ]:
all_price_df.one_hot_win.sum()

In [ ]:
all_price_df['correct_over_fav'] = all_price_df['correct'] - all_price_df['favorite_correct']

In [ ]:
all_price_df['correct_over_fav'].sum()

In [ ]:
all_price_df.query('correct == 1')['prices'].mean()

In [ ]:
all_price_df.query('favorite_correct == 1')['prices'].mean()

In [ ]:
print(f"{all_price_df['profit_model < 30'].sum()=}")
print(f"{all_price_df2['profit_model < 30'].sum()=}")

In [ ]:
all_price_df['dt'] = pd.to_datetime(all_price_df['date']).dt.date
all_price_df = all_price_df.sort_values('dt',ascending=True)

In [ ]:
all_price_df = all_price_df2

In [ ]:
all_price_df['lay_profit_model'] = all_price_df['lay_amount_model']-all_price_df['onehot_win']*all_price_df['lay_amount_model']*all_price_df['prices']

In [ ]:
all_price_df['lay_profit_model'].sum()

In [ ]:
bet_all_df = all_price_df[all_price_df['bet_amount_model'] >= 0.6]

In [ ]:
bet_all_df['bet_under_30'] = np.where(bet_all_df['prices'] < 30,1,0)

In [ ]:
bet_all_df['bet_under_30'].mean()

In [ ]:
all_price_df.to_excel(f'model_all_price/{epoch}{model_name} - val-all_price_df.xlsx')

In [ ]:
model_name  = 'sage-blaze-232'
epoch = 200
all_price_df = pd.read_feather(f'model_all_price/{epoch}{model_name} - all_price_df.fth')
all_price_df
all_price_df2 = pd.read_feather(f'model_all_price/{epoch}{model_name} - val_all_price_df.fth')
all_price_df = pd.concat([all_price_df, all_price_df2])

In [ ]:
all_price_df.date.min()


In [ ]:
all_price_df.columns

In [ ]:
all_price_df.raceID.nunique()

In [ ]:
all_price_df.race_num.astype(int).value_counts(sort=False)

In [ ]:
all_price_df.to_parquet(f'model_all_price/{model_name} - all_price_df.pqt')

In [ ]:
all_price_df.shape

In [ ]:
all_price_df.to_excel(f'model_all_price/{model_name} - all_price_df.xlsx')

In [ ]:
model_name = 'faithful-sunset-63'
all_price_df_nz = pd.read_parquet(f'model_all_price/{model_name} - all_price_df.pqt')
all_price_df = all_price_df_nz

In [ ]:
import matplotlib.pyplot as plt

# Filter the dataframe where 'bet_amount_model' > 0.5
filtered_df = all_price_df[all_price_df['bet_amount_model'] > 0.8]
filtered_df = filtered_df[filtered_df['round_price'] < 100]
weights = np.ones_like(filtered_df['round_price']) / len(filtered_df['round_price'])

# Plot histogram with weights
plt.hist(filtered_df['round_price'], weights=weights, bins=50, edgecolor='black')

plt.xlabel('Round Price')
plt.ylabel('Percentage of bet_amount_model > 0.5')
plt.title('Histogram of percentages of bet_amount_model > 0.5 against round_price')
plt.show()

In [ ]:
filtered_df['round_price'].mean()

In [ ]:
filtered_df

In [ ]:
filtered_df = all_price_df[all_price_df['lay_amount_model'] > 0.8]
filtered_df = filtered_df[filtered_df['round_price'] < 30].reset_index(drop=True).copy()
average_price_per_day = filtered_df.groupby('date')['round_price'].mean()

# Plot histogram
plt.hist(average_price_per_day, bins=30, edgecolor='black')

plt.xlabel('Average Price')
plt.ylabel('Frequency')
plt.title('Histogram of average prices per day')
plt.show()

In [ ]:
all_price_df = pd.concat([all_price_df, all_price_df_nz])

In [ ]:
all_price_df = pd.read_excel(f'model_all_price/{model_name} - all_price_df.xlsx')

In [ ]:
price_df  = all_price_df.groupby('round_price')[['lay_amount_model', 'lay_profit_model','bet_amount_model', 'profit_model']].sum().cumsum().reset_index()


In [ ]:
import plotly.graph_objects as go
price_df['roi'] = price_df['profit_model'] / price_df['bet_amount_model']
price_df = price_df.query('round_price < 100.0')
# Create a trace for ROI
trace_roi = go.Scatter(
    x = price_df['round_price'],
    y = price_df['profit_model'],
    mode = 'lines',
    name = 'Profit Model'
)

data = [trace_roi]

# Define layout
layout = go.Layout(
    title = 'Interactive Plot',
    xaxis = dict(title = 'Round Price'),
    yaxis = dict(title = 'Amount'),
)

# Define the figure
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()


In [ ]:
all_price_df['dt'] = pd.to_datetime(all_price_df['date']).dt.date
all_price_df = all_price_df.sort_values('dt',ascending=True)

In [ ]:
model_name  = 'fiery-haze-173'
all_price_df2 = pd.read_feather(f'model_all_price/{epoch}{model_name} - val_all_price_df.fth')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import glob
import re
model_name  = 'fiery-haze-173'
files = glob.glob(f'model_all_price/*{model_name}* - val_all_price_df.fth')

# Create a figure and a single axis
fig, ax1 = plt.subplots(figsize=(10, 6))

colors = sns.color_palette('husl', len(files))
for i,file in enumerate(files):
    all_price_df = pd.read_feather(file)
    all_price_df['dt'] = pd.to_datetime(all_price_df['date']).dt.date

    date = all_price_df.query('round_price <= 24.0').copy()

    date['bet_amount_model'] = np.where(date['bet_amount_model'] > 0.1, date['bet_amount_model'], 0)*10*2
    date['profit_model'] = np.where(date['bet_amount_model'] > 0.1, date['profit_model'], 0)*10*2

    date = date.groupby('dt',sort=False)[['bet_amount_model', 'profit_model','round_price']].sum().cumsum().reset_index()
    date['roi'] = date['profit_model'] / date['bet_amount_model']
    epoch_number = re.search(r'\d+', file).group()
    # Plot the 'Profit Model' on the first axis
    color = colors[i]
    ax1.plot(date['dt'], date['profit_model'], label=f'Profit Model {epoch_number}', color=color)
    ax1.set_xlabel('Round Price')
    ax1.set_ylabel('Amount', color=color)

    # Create a second axis that shares the same x-axis
    # ax2 = ax1.twinx()

    # Plot the 'roi' on the second axis
    color = sns.color_palette()[1]
    # ax2.plot(date['dt'], date['roi'], label=f'roi {file}', color=color)
    # ax2.set_ylabel('roi', color=color)

# Show the legend
fig.legend(loc="upper left")

# Show the plot
plt.show()

In [ ]:
all_price_df.columns

In [ ]:
price_df  = all_price_df.query('round_price < 30').groupby('raceID')[['bet_amount_model', 'profit_model']].sum()

In [ ]:
price_df

In [ ]:
all_price_df.query('round_price < 30')['profit_model'].mean()

In [ ]:
all_price_df['roi'] = all_price_df['profit_model'] / all_price_df['bet_amount_model']

In [ ]:
price_df  = all_price_df.query('round_price < 30').groupby('raceID')[['bet_amount_model', 'profit_model']].sum()

In [ ]:
price_df

In [ ]:
price_df.mean()

In [ ]:
import scipy.stats as stats
model_name  = 'fiery-haze-173'
# file
# Concatenate all profit_model data from all files
# all_profits =  pd.read_parquet('model_all_price/fiery-haze-173 - all_price_df.pqt')['profit_model']
# all_price_df = price_df
all_price_df['roi'] = all_price_df['profit_model'] / all_price_df['bet_amount_model']
# all_price_df = all_price_df.query('bet_amount_model > 0.10 and prices < 30').copy()

all_profits = all_price_df['roi']
all_profits.dropna(inplace=True)

# Perform the t-test
t_stat, p_value = stats.ttest_1samp(all_profits, 0.15)

print(f'T-statistic: {t_stat}')
print(f'P-value: {p_value}')

In [ ]:
all_profits

In [ ]:

all_profits = all_price_df.groupby('raceID')[['bet_amount_model','profit_model']].sum()
all_profits['roi'] = all_profits['profit_model'] / all_profits['bet_amount_model']
all_profits = all_profits['roi']

In [ ]:
import numpy as np
all_profits 
std_dev = all_profits.std()
num = all_profits.count()
print(f'Standard deviation: {std_dev}')

Z = 1.96  # z-score for 95% confidence level
E = 0.1  # desired margin of error

# Calculate the sample size
n = (Z**2 * std_dev**2) / E**2
n = np.ceil(n)  # round up to the nearest whole number because you can't have a fraction of a sample

print(f'Sample size: {n}, num = {num}')

In [ ]:
win_df = all_price_df.query('round_price < 30.0').groupby('dt')['profit_model'].sum().reset_index()
win_df['win'] = np.where(win_df['profit_model'] > 0, 1, 0)
win_df['win'].value_counts()

In [ ]:
(Z**2 * std_dev**2)

In [ ]:
all_profits.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the seaborn color palette as the default matplotlib color palette
sns.set_palette('husl')

# Your existing code
date = all_price_df.query('round_price < 31.0').copy()

date['bet_amount_model'] = np.where(date['bet_amount_model'] > 0.1, date['bet_amount_model'], 0)*10*2
date['profit_model'] = np.where(date['bet_amount_model'] > 0.1, date['profit_model'], 0)*10*2

date = date.groupby('dt',sort=False)[['bet_amount_model', 'profit_model','round_price']].sum().cumsum().reset_index()
date['balance'] = date['profit_model']
date['adj_bet'] = np.where(date['balance']*0.05>20, date['balance']*0.05,20)
date['adj_profit'] = date['adj_bet']*date['profit_model']
date['roi'] = date['profit_model'] / date['bet_amount_model']
date['adj_roi'] = date['adj_profit'] / (date['adj_bet'].cumsum()*20)
# Create a figure and a single axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the 'Profit Model' on the first axis
color = sns.color_palette()[0]
ax1.plot(date['dt'], date['adj_profit'], label='Profit Model', color=color)
ax1.set_xlabel('Round Price')
ax1.set_ylabel('Amount', color=color)

color = sns.color_palette()[2]
ax1.plot(date['dt'], date['profit_model'], label='Profit Model', color=color)
ax1.set_xlabel('Round Price')
ax1.set_ylabel('Amount', color=color)


# Create a second axis that shares the same x-axis
ax2 = ax1.twinx()

# Plot the 'roi' on the second axis
color = sns.color_palette()[1]
ax2.plot(date['dt'], date['adj_roi'], label='roi', color=color)
ax2.set_ylabel('roi', color=color)

# Show the legend
fig.legend(loc="upper left")

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the seaborn color palette as the default matplotlib color palette
sns.set_palette('husl')

# Your existing code
date = all_price_df.query('round_price < 31.0').copy()
start_date = '2023-07-01'
start_date = pd.to_datetime(start_date).date()
date = date.query('dt >= @start_date').copy()
# date['bet_amount_model'] = date['lay_amount_model']# + date['bet_amount_model']
# date['profit_model'] = date['lay_profit_model']# + date['profit_model']
# date['bet_amount_model'] = np.where(date['bet_amount_model'] > 0.1, date['bet_amount_model'], 0)*1
# date['profit_model'] = np.where(date['bet_amount_model'] > 0.1, date['profit_model'], 0)*1
# date['profit_model'] = np.where(date['profit_model']>0,date['profit_model'],date['profit_model'])
date = date.groupby('dt',sort=False)[['bet_amount_model', 'profit_model','round_price']].sum().cumsum().reset_index()
date['roi'] = date['profit_model'] / date['bet_amount_model']

# Create a figure and a single axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Create a figure and a single axis
fig, ax1 = plt.subplots(figsize=(10, 6))
prices = [3,5,10,15,20,25,30,31,32,35,50,100,10000]
colors = sns.color_palette('husl', len(prices))
for i,price in enumerate(prices):
    print(f"{price=}")
    date = all_price_df.query('round_price < @price').copy()
    start_date = '2023-07-01'
    start_date = pd.to_datetime(start_date).date()
    date = date.query('dt >= @start_date').copy()
    date = date.groupby('dt',sort=False)[['bet_amount_model', 'profit_model','round_price']].sum().cumsum().reset_index()
    # Plot the 'Profit Model' on the first axis
    color = colors[i]
    ax1.plot(date['dt'], date['profit_model'], label=f'Profit Model {price}', color=color)
    ax1.set_xlabel('Round Price')
    ax1.set_ylabel('Amount', color=color)

    # Create a second axis that shares the same x-axis

    ax2 = ax1.twinx()

    # Plot the 'roi' on the second axis
    color = sns.color_palette()[1]
    # ax2.plot(date['dt'], date['roi'], label='roi', color=color)
    # ax2.set_ylabel('roi', color=color)

    # Show the legend
fig.legend(loc="upper left")

    # Show the plot
plt.show()

In [ ]:
#Testing difference in data between server and local
local = pd.read_feather('./data/gru_inputs_kitchen_sink_updated_recent.fth')
server = pd.read_feather('./data/testing new outs simple 2024-03-24_NZ.fth')

In [ ]:
local = local.rename({'stats':'stats_local','dog_id':'dogid'}, axis=1)
server = server.rename({'stats':'stats_server'}, axis=1)

In [ ]:
local.query('raceid=="1009424823"')

In [ ]:
local = local[['dogid','raceid','stats_local','prev_race_date']]
merged = server.merge(local, on=['dogid','raceid'], how='left')

In [ ]:
local.query('dogid == "760119322"')

In [ ]:
merged['stats_diff'] = merged.apply(lambda x: (x['stats_local'] - x['stats_server']).sum(), axis=1)

In [ ]:
dogid = "505983450"
stats_local = merged.query('dogid == @dogid')['stats_local'].to_list()[0]
stats_server = merged.query('dogid == @dogid')['stats_server'].to_list()[0]


In [ ]:
stats_local.round(2).tolist()

In [ ]:
delta = pd.DataFrame([stats_server.round(2)[6:],stats_local.round(2)[6:]],columns=eval(merged['stats_cols'][0]))

In [ ]:
local = local.query('state == "NZ"')

In [ ]:
local

In [ ]:
server['stats_cols'][0]

In [ ]:
df = df[[col for col in df.columns if '2' not in col and '<' not in col]]

In [ ]:
df['bet_kelly_seq'] = np.where(
    df['prices']<30,
    df['seq_kelly_ratio']*100,
    0
)


df['profit_kelly_sq'] = np.where(
    df['onehot_win'],
    df['bet_kelly_seq']  * (df['prices'] - 1) * 0.95,
    -1 * df['bet_kelly_seq']
)

In [ ]:
df_30 = df[df['prices']<30]

In [ ]:
df_30['profit_kelly_sq'].sum()/df_30['bet_kelly_seq'].sum()

In [ ]:
df_30['profit_kelly'].sum()/df_30['bet_amount_kelly'].sum()

In [ ]:
json_path = 'wandb_jsons/dancing-snake-80' 
files = os.listdir(json_path)
file_paths = [json_path + '/' + file for file in files]

for i,file in enumerate(file_paths):
    with open(file, 'r') as f:
        json_obj = json.load(f)

    # Create a DataFrame from the dictionary
    if i == 0:
        data = pd.DataFrame(data=json_obj["data"], columns=json_obj["columns"])
        data['step'] = i
    else:
        df = pd.DataFrame(data=json_obj["data"], columns=json_obj["columns"])
        df['step'] = i
        data = pd.concat([data, df])

In [ ]:
data

In [ ]:
df.filter

In [ ]:
df = data
df = data.query('onehot_win ==1')
# df = df.head(1000)

# Create a scatter plot for each step
# Create a scatter plot for each step
# Create a scatter plot for each step using Scattergl for WebGL
traces = [go.Scattergl(
    x=df['loss'][df['step'] == step].head(100),
    y=df['mutual_info'][df['step'] == step].head(100),
    mode='markers',
    visible=False,
    name=f'Step {step}'
) for step in df['step'].unique()]

# Make the first trace visible
traces[0]['visible'] = True

x_range = [df['loss'].min(), df['loss'].max()]
y_range = [df['mutual_info'].min(), df['mutual_info'].max()]

# Create a layout with a slider
layout = go.Layout(
    title='Loss vs Mutual Info',
    xaxis=dict(title='Loss'),
    yaxis=dict(title='Mutual Info'),
    sliders=[dict(
        active=0,
        currentvalue={"prefix": "Step: "},
        pad={"t": 50},
        steps=[dict(method='update',
                    args=[{'visible': [s == step for s in df['step'].unique()]}],
                    label=str(step)) for step in df['step'].unique()]
    )]
)

# Create a Figure
fig = go.Figure(data=traces, layout=layout)

# Display the figure
fig.show()